In [ ]:
"""
Use fakeit in xspec
1. model and fakeit
2. fit it with xspec, check the constraints
3. 

needs to be inst variance

"""

# conditions

1T 
1keV
abun 0.3
norm 1e-5

2T
1keV and 3keV
abun 0.3
norm 1e-5

GDEM
T 1.5
sigma 1 
norm 1e-5

In [2]:
from astropy.io import fits
import numpy as np
from glob import glob

# mincounts from fakeit

In [26]:
# expscale 3 can be constrained the Z as 0.35+/-0.1
# calculate the minimum counts for every inst

datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID9647/fitting/R500_01/fake_snr'
mdl = '2T'
app = 'expsc3'
mincts_dict = {}
for inst in ['mos1', 'mos2', 'pn']:
    with fits.open(f'{datapath}/fake_{mdl}_{inst}_R500_01_{app}.fak') as f:
        mincts_dict[inst] = np.sum(f[1].data['COUNTS'])

print(mincts_dict)

{'mos1': 84, 'mos2': 89, 'pn': 349}


# counts from data

## 1 just counts 

In [14]:
SRCNAME1 = 'ID9647'
SRCNAME2 = 'SDSSTG9647'
REGNAME = 'R500_01'

erange = (0.5,4)
datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/{SRCNAME1}/fitting/{REGNAME}'

datacts_dict = {}
for inst in ['mos1', 'mos2']:
    with fits.open(glob(f'{datapath}/{inst}*{SRCNAME2}_{REGNAME}.rmf')[0]) as f:
        emin = f[2].data['E_MIN']
        efil = (emin>erange[0]) & (emin<erange[1])
    with fits.open(glob(f'{datapath}/{inst}*-obj-*{SRCNAME2}_{REGNAME}.pi')[0]) as f:
        cts = np.sum(f[1].data['COUNTS'][efil])
    with fits.open(glob(f'{datapath}/{inst}*-back-*{SRCNAME2}_{REGNAME}.pi')[0]) as f:
        if inst == 'pn':
            bkg = np.sum(f[1].data['RATE'][efil]) * float(f[1].header['EXPOSURE'])
        else:
            bkg = np.sum(f[1].data['COUNTS'][efil])
    datacts_dict[inst] = cts - bkg
print(datacts_dict)

371
431


IndexError: list index out of range